## Langchain/LangGraph with Nova Models

To aid in the development of applications with Nova models; Langchain support has been added for multimodal and agentic workflows.

## Prerequisites

#### Install python libraries

Run the cells in this section to install the packages needed by the notebooks in this workshop. ⚠️ You will see pip dependency errors, you can safely ignore these errors. ⚠️

_IGNORE ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts._

In [ ]:
%pip install -q --upgrade langchain langchain_aws langchain_community matplotlib 

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Model Invocation

#### Text Understanding

In [ ]:
LITE_MODEL_ID = "us.amazon.nova-2-lite-v1:0"

## ChatBedrockConverse() API allows you to use Bedrock hosted LLM and create chat type interaction

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

llm = ChatBedrockConverse(
    model_id=LITE_MODEL_ID,
    temperature=0.7
)

messages = [
    ("system", "Provide three alternative song titles for a given user title"),
    ("user", "Teardrops on My Guitar"),
]

response = llm.invoke(messages)
print(f"Request ID: {response.id}")
response.pretty_print()


# Here we can pass the chat history to the model to ask follow up questions
multi_turn_messages = [
    *messages,
    response,
    HumanMessage(content="Select your favorite and tell me why"),
]

response = llm.invoke(multi_turn_messages)
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Image Understanding

You are able to pass various media types to the model

In [ ]:
from IPython.display import Image

image_path = "media/sunset.png"
Image(filename=image_path)

In [ ]:
import base64

from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage

llm = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=0.7
)

with open(image_path, "rb") as image_file:
    binary_data = image_file.read()

message = HumanMessage(
    content=[
        {"image": {"format": "png", "source": {"bytes": binary_data}}},
        {"text": "Provide a summary of this photo"},
    ]
)

response = llm.invoke([message])
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Video Understanding

In [ ]:
video_path = "media/the-sea.mp4"

In [ ]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=0.7
)

with open(video_path, "rb") as video_file:
    binary_data = video_file.read()


message = HumanMessage(
    content=[
        {"video": {"format": "mp4", "source": {"bytes": binary_data}}},
        {"type": "text", "text": "Describe the following video"},
    ]
)

response = llm.invoke([message])
print(f"\n\nRequest ID: {response.id}")
response.pretty_print()

#### Streaming

Streaming is also supported

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=0.7
)

chain = llm | StrOutputParser()

messages = [
    SystemMessage(content="You are an author with experience writing creative novels"),
    HumanMessage(
        content="Write an outline for a novel about a wizard named Theodore graduating from college"
    ),
]

for chunk in chain.stream(messages):
    print(chunk, end="")

## Agent Workflows

The Nova model is capable of handling tool calling and agentic workflows.

#### Binding Tools

When using a model for tool calling you can take advantage of the bind tools method. This will pass a formatted tool config to the model. We recommend when taking advantage of tool calling or agentic workflows to use greedy decoding values. This means temperature=1, topP=1, topK=1

In [ ]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import ToolMessage
import json

def multiply_numbers(a: float, b: float) -> float:
    """Multiply two numbers
        a: float, "First number to multiply"
        b: float, "Second number to multiply"
        returns multiplication of a and b (a * b)
    """
    return a * b
    

tools = [multiply_numbers]

llm_with_tools = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
).bind_tools(tools)

messages = [("user", "What is 8*9.5")]
response = llm_with_tools.invoke(messages)
print("=" * 80)
print("[Model Response]\n")
print(json.dumps(response.content, indent=2))
print("=" * 80)
print("\n[Tool Calls]\n")
print(json.dumps(response.tool_calls, indent=2))
print("=" * 80)

# Handle the Tool Call and Execute the Tool
if response.tool_calls:
    for tool_call in response.tool_calls:
        if tool_call['name'] == "multiply_numbers":
            result = multiply_numbers(tool_call['args']['a'], tool_call['args']['b'])
            # Send the tool's output back to the model
            tool_message = ToolMessage(content=str(result), tool_call_id=tool_call['id'])
            follow_up_response = llm_with_tools.invoke(messages + [response]+ [tool_message])
            print("=" * 80)
            print(follow_up_response.content)
            print("=" * 80)

#### Tool Calling Agents

For full workflows you can take advantage of custom parsers that will intercept outputs of the stream and allow you to invoke tools

In [ ]:
from langchain import tools
from langchain.tools import tool
from langchain_classic.agents import AgentExecutor 
from langchain_classic.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrockConverse

In [ ]:
## AgentExecutor() api is being deprecated. Use with Caution.
## Recommended pattern is to use LangGraph 
 
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

tools = [multiply]

llm_with_tools = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
).bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is 2*2?"})

## Structured Output

Structured output is a great way to force the model to return in a specific way. We use Greedy Decoding params here for more determinist results (Temperature = 1, Top P = 1, Top K = 1)

In [ ]:
from pydantic import BaseModel, Field
from langchain_aws import ChatBedrockConverse

class Joke(BaseModel):
    """A joke to respond to the user"""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")

llm = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
)
structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

## A very simple Langchain Agent

In [ ]:
from langchain.agents import create_agent
import json
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"Test Response: It's always sunny in {city}!"

MODEL_ID = "amazon.nova-2-lite-v1:0"
agent = create_agent(
    model=MODEL_ID,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response['messages'][3].content

## Create an agent using LangGraph 

In [ ]:
# Import the required libraries and methods
from typing import List, Literal
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END
from pydantic import BaseModel, Field
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model=LITE_MODEL_ID,
    temperature=1,
    top_p=1,
    additional_model_request_fields={
        "inferenceConfig": {
            "topK": 1
        }
    },
)

@tool
def get_weather(city: str) -> list:
    """ get the current weather"""
    if str is not None:
        """Get weather for a given city."""
        return f"Custom Tool Response: It's always rainy in {city}!"
    else:
        return "Weather Data Not Found"
    
tools = [get_weather]
tool_node = ToolNode(tools)

llm_with_tools = llm.bind_tools(tools)

def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def call_tools(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
## Initialize nodes and edges

workflow = StateGraph(MessagesState)

# add a node named LLM, with call_model function. This node uses an LLM to make decisions based on the input given
workflow.add_node("LLM", call_model)

# Our workflow starts with the LLM node
workflow.add_edge(START, "LLM")

# Add a tools node
workflow.add_node("tools", tool_node)

# Add a conditional edge from LLM to call_tools function. It can go tools node or end depending on the output of the LLM. 
workflow.add_conditional_edges("LLM", call_tools)

# tools node sends the information back to the LLM
workflow.add_edge("tools", "LLM")

agent = workflow.compile()

In [ ]:
from IPython.display import Image, display
# Generate the image 
display(Image(agent.get_graph().draw_mermaid_png()))


In [ ]:
for chunk in agent.stream(
    {"messages": [("user", "Will it rain in Seattle today?")]},
    stream_mode="values",):
    chunk["messages"][-1].pretty_print()